In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
import os
import zipfile
import datetime

In [ ]:
def get_options(symbol, expiry=None):
    """
    Fetch options chain for a symbol.
    - If expiry=None → returns all available expiry dates
    - If expiry is provided → returns (calls, puts) DataFrames
    """
    ticker = yf.Ticker(symbol)
    expirations = ticker.options

    if expiry is None:
        return expirations

    if expiry not in expirations:
        raise ValueError(f"Invalid expiry {expiry}. Choose from: {expirations}")

    chain = ticker.option_chain(expiry)
    return chain.calls, chain.puts

In [ ]:
symbol = input("Enter ticker symbol (e.g., SPY, TSLA): ").upper()
expirations = get_options(symbol)
print("Available expirations:")
for i, exp in enumerate(expirations):
    print(f"{i}: {exp}")

choice = input("Type '1' for single expiry or '5' for five expiries: ").strip()

if choice == "1":
    idx = int(input("Enter the index of the expiry you want: "))
    chosen = [expirations[idx]]
elif choice == "5":
    idxs = input("Enter 5 indices separated by commas (e.g., 0,1,2,3,4): ")
    idxs = [int(x.strip()) for x in idxs.split(",")]
    chosen = [expirations[i] for i in idxs]
else:
    raise ValueError("Invalid choice. Type 1 or 5.")

print(f"Chosen expirations: {chosen}")

Enter ticker symbol (e.g., SPY, TSLA): spy
Available expirations:
0: 2025-09-19
1: 2025-09-22
2: 2025-09-23
3: 2025-09-24
4: 2025-09-25
5: 2025-09-26
6: 2025-09-30
7: 2025-10-03
8: 2025-10-10
9: 2025-10-17
10: 2025-10-24
11: 2025-10-31
12: 2025-11-21
13: 2025-11-28
14: 2025-12-19
15: 2025-12-31
16: 2026-01-16
17: 2026-01-30
18: 2026-02-20
19: 2026-02-27
20: 2026-03-20
21: 2026-03-31
22: 2026-06-18
23: 2026-06-30
24: 2026-09-18
25: 2026-12-18
26: 2027-01-15
27: 2027-12-17
Type '1' for single expiry or '5' for five expiries: 1
Enter the index of the expiry you want: 0
Chosen expirations: ['2025-09-19']


In [ ]:
# Fetch all data for chosen expiries
all_calls = []
all_puts = []

for expiry in chosen:
    print(f"Fetching {symbol} {expiry}...")
    calls, puts = get_options(symbol, expiry)
    calls['expiry'] = expiry
    puts['expiry'] = expiry
    all_calls.append(calls)
    all_puts.append(puts)

# Combine into single DataFrames
df_calls = pd.concat(all_calls, ignore_index=True)
df_puts = pd.concat(all_puts, ignore_index=True)

Fetching SPY 2025-09-19...


In [ ]:
# 🔹 Cell 6: Preprocess Options + Dynamic Spot Range

def preprocess_options(df_calls, df_puts):
    # Standardize column names
    df_calls.columns = [c.lower().strip() for c in df_calls.columns]
    df_puts.columns = [c.lower().strip() for c in df_puts.columns]

    # Rename columns to short names
    df_calls = df_calls.rename(columns={
        "openinterest":"oi_call",
        "volume":"vol_call",
        "impliedvolatility":"iv_call"
    })

    df_puts = df_puts.rename(columns={
        "openinterest":"oi_put",
        "volume":"vol_put",
        "impliedvolatility":"iv_put"
    })

    # Keep only needed columns
    df_calls = df_calls[["strike","oi_call","vol_call","iv_call","lasttradedate"]]
    df_puts  = df_puts[["strike","oi_put","vol_put","iv_put","lasttradedate"]]

    # Merge on strike
    df = pd.merge(df_calls, df_puts, on="strike", suffixes=("_call","_put"))

    # Ensure numeric
    for col in ["strike","iv_call","oi_call","iv_put","oi_put"]:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

    # Fix IV scaling + fallback
    def fix_iv(iv):
        if iv > 1:   # percent -> decimal
            return iv/100
        return max(iv, 0.20)

    df["iv_call"] = df["iv_call"].apply(fix_iv)
    df["iv_put"]  = df["iv_put"].apply(fix_iv)

    return df

# Apply preprocessing
df = preprocess_options(df_calls, df_puts)

# Prompt user for spot price
spot = float(input("Enter current spot price: "))

# Prompt user for strike range above/below spot
range_strikes = int(input("Enter number of strikes above and below spot to include: "))

# Filter strikes within user-defined range
df = df[(df["strike"] >= spot - range_strikes) & (df["strike"] <= spot + range_strikes)]

# Debug: check filtered strikes
print(f"Filtered strikes ±{range_strikes} around spot {spot}")
print(df.head(20))

Enter current spot price: 662.26
Enter number of strikes above and below spot to include: 20
Filtered strikes ±20 around spot 662.26
     strike  oi_call  vol_call   iv_call        lasttradedate_call   oi_put  \
162   643.0     2886   15534.0  0.475835 2025-09-18 20:11:30+00:00   7471.0   
163   644.0     2714   15710.0  0.582768 2025-09-18 20:13:02+00:00   6463.0   
164   645.0    24888  162601.0  0.562016 2025-09-18 20:11:19+00:00  45820.0   
165   646.0     2959   15566.0  0.531499 2025-09-18 20:07:49+00:00  21395.0   
166   647.0     4928   30542.0  0.468755 2025-09-18 20:07:54+00:00  33590.0   
167   648.0     6889   38477.0  0.422125 2025-09-18 20:13:11+00:00   9460.0   
168   649.0     6796   42878.0  0.476690 2025-09-18 20:13:02+00:00   5928.0   
169   650.0    52013  391466.0  0.258552 2025-09-18 20:13:55+00:00  44585.0   
170   651.0     3799   25462.0  0.386725 2025-09-18 20:07:49+00:00   9679.0   
171   652.0     5397   42102.0  0.414557 2025-09-18 20:14:23+00:00  14622.0  

In [ ]:
# 🔹 Cell 7: Calculate Gamma, Dollar GEX, and Aggregate by Strike

import numpy as np
from scipy.stats import norm

# Parameters
r = 0.05
T = 1/252  # 0DTE

# Black-Scholes gamma
def bs_gamma(S,K,T,r,sigma):
    if T <= 0 or sigma <= 0:
        return 0
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    return norm.pdf(d1)/(S*sigma*np.sqrt(T))

# Calculate gamma for each strike
df["gamma_call"] = df.apply(lambda row: bs_gamma(spot, row["strike"], T, r, row["iv_call"]), axis=1)
df["gamma_put"]  = df.apply(lambda row: bs_gamma(spot, row["strike"], T, r, row["iv_put"]), axis=1)

# Dollar GEX
df["call_gex"] = df["gamma_call"] * df["oi_call"] * 100 * spot
df["put_gex"]  = -df["gamma_put"] * df["oi_put"] * 100 * spot

# Aggregate by strike
df_total = df.groupby("strike").agg({
    "call_gex":"sum",
    "put_gex":"sum",
    "oi_call":"sum",
    "oi_put":"sum"
}).reset_index()

df_total["net_gex"] = df_total["call_gex"] + df_total["put_gex"]
df_total["abs_gex"] = df_total["call_gex"].abs() + df_total["put_gex"].abs()
df_total["total_oi"] = df_total["oi_call"] + df_total["oi_put"]

# Compute total absolute gamma (magnitude) and total net gamma (directional)
total_gamma_abs = (df_total["call_gex"].abs() + df_total["put_gex"].abs()).sum()
total_net_gamma = df_total["net_gex"].sum()

In [ ]:
# 🔷 Cell: Power Zone (insert after df_total is computed and filtered, before plotting)

# Requirements: df_total must already exist with columns:
# "strike", "abs_gex", "net_gex", "total_oi"
# and `spot` must exist.

# Fallback safety: ensure numeric and present columns
for c in ["strike","abs_gex","total_oi"]:
    if c not in df_total.columns:
        df_total[c] = 0
    df_total[c] = pd.to_numeric(df_total[c], errors="coerce").fillna(0)

# 1) Choose weight: prefer abs_gex (shows hedging concentration).
#    But if abs_gex sums to almost zero (premarket/low activity), fall back to total_oi.
abs_gex_sum = df_total["abs_gex"].sum()
oi_sum = df_total["total_oi"].sum()

if abs_gex_sum > 1e-6:
    primary_weights = df_total["abs_gex"].copy()
    weight_source = "abs_gex"
else:
    primary_weights = df_total["total_oi"].copy()
    weight_source = "total_oi"

# Mix both signals to be more robust: final weight = abs_gex + alpha * total_oi
alpha = 0.25   # small contribution from OI; tweakable
weights = primary_weights + alpha * df_total["total_oi"]

# If weights are all zero, make uniform weights (avoid division by zero)
if weights.sum() == 0:
    weights = pd.Series(1.0, index=df_total.index)

# Compute weighted center (power zone center) and weighted std (band width)
power_center = (df_total["strike"] * weights).sum() / weights.sum()
# weighted variance:
var = ((weights * (df_total["strike"] - power_center)**2).sum()) / weights.sum()
power_std = np.sqrt(var)

# Define bands (you can change k for wider/narrower band)
k = 1.0   # 1 std dev band (common choice for mean-reversion "power zone")
power_lower = power_center - k * power_std
power_upper = power_center + k * power_std

# Power score: concentration of weight inside the band (0..1)
inside_mask = (df_total["strike"] >= power_lower) & (df_total["strike"] <= power_upper)
power_score = weights[inside_mask].sum() / weights.sum()

# Extra helpful metrics
largest_abs_gex_strike = df_total.loc[df_total["abs_gex"].idxmax(), "strike"] if not df_total.empty else None
largest_net_gex_strike = df_total.loc[df_total["net_gex"].abs().idxmax(), "strike"] if not df_total.empty else None

# Print results (human-friendly)
print("🔷 POWER ZONE METRICS")
print(f"- Weight source used primarily: {weight_source} (+ alpha*total_oi={alpha})")
print(f"- Power Zone center  : {power_center:.2f}")
print(f"- Power Zone band    : [{power_lower:.2f} , {power_upper:.2f}]  (k={k} stddev, std={power_std:.2f})")
print(f"- Power score (mass inside band): {power_score:.2%}")
print(f"- Strike with highest abs gamma: {largest_abs_gex_strike}")
print(f"- Strike with largest net gex magnitude: {largest_net_gex_strike}")

# Interpretation guideline (quick)
if power_score > 0.6:
    interp = "STRONG mean-reversion expected toward power center today."
elif power_score > 0.35:
    interp = "MODERATE mean-reversion potential."
else:
    interp = "WEAK mean-reversion signal — more free to trend/breakout."

print(f"\nInterpretation: {interp}")

# Optional: add to your existing Plotly fig if you already created "fig".
# This snippet does not create a new fig; it will add visuals to `fig` if present.
# Add this block below only if you want the shading/line on the chart:
add_visuals_to_fig = True  # set False if you do NOT want chart edits here
if add_visuals_to_fig:
    try:
        # light shaded band and center line
        fig.add_vrect(x0=power_lower, x1=power_upper,
                      fillcolor="rgba(128,0,128,0.18)", line_width=0, layer="below")
        fig.add_vline(x=power_center, line=dict(color="magenta", width=2, dash="dot"),
                      annotation_text="Power Center", annotation_position="bottom right")
        # Add a small annotation with the power score
        fig.add_annotation(xref="paper", yref="paper",
                           x=0.01, y=0.92,
                           text=f"Power score: {power_score:.2%}",
                           showarrow=False, font=dict(color="white"), bgcolor="rgba(0,0,0,0.4)")
        # If you call fig.show() afterwards it will include the Power Zone visuals.
    except NameError:
        # fig doesn't exist in this scope — that's fine, we've still computed the numbers above.
        pass

🔷 POWER ZONE METRICS
- Weight source used primarily: abs_gex (+ alpha*total_oi=0.25)
- Power Zone center  : 659.20
- Power Zone band    : [651.12 , 667.28]  (k=1.0 stddev, std=8.08)
- Power score (mass inside band): 60.52%
- Strike with highest abs gamma: 660.0
- Strike with largest net gex magnitude: 670.0

Interpretation: STRONG mean-reversion expected toward power center today.


In [ ]:
# 🔹 Cell 8: Plot GEX Dashboard (Net GEX, Abs GEX, OI, Spot, Max OI)

import plotly.graph_objects as go

# Color Net GEX by sign
colors = ["green" if val >= 0 else "red" for val in df_total["net_gex"]]

fig = go.Figure()

# Net GEX bars
fig.add_trace(go.Bar(
    x=df_total["strike"],
    y=df_total["net_gex"],
    name="Net GEX",
    marker_color=colors,
    yaxis="y1",
    hovertemplate='🟢 Net GEX: %{y:.0f}<extra></extra>'
))

# Abs GEX line
fig.add_trace(go.Scatter(
    x=df_total["strike"],
    y=df_total["abs_gex"],
    mode="lines+markers",
    name="Abs GEX",
    line=dict(color="purple", width=2),
    yaxis="y2",
    hovertemplate='🟣 Abs GEX: %{y:.0f}<extra></extra>'
))

# Call OI markers
fig.add_trace(go.Scatter(
    x=df_total["strike"],
    y=df_total["oi_call"],
    mode="markers",
    name="Call OI",
    marker=dict(color="green", size=8, symbol="circle"),
    yaxis="y1",
    hovertemplate='🟢 Call OI: %{y:.0f}<extra></extra>',
    visible="legendonly"
))

# Put OI markers
fig.add_trace(go.Scatter(
    x=df_total["strike"],
    y=df_total["oi_put"],
    mode="markers",
    name="Put OI",
    marker=dict(color="red", size=8, symbol="circle"),
    yaxis="y1",
    hovertemplate='🔴 Put OI: %{y:.0f}<extra></extra>',
    visible="legendonly"
))

# Total OI markers
fig.add_trace(go.Scatter(
    x=df_total["strike"],
    y=df_total["total_oi"],
    mode="markers",
    name="Total OI",
    marker=dict(color="white", size=8, symbol="circle"),
    yaxis="y1",
    hovertemplate='⚪ Total OI: %{y:.0f}<extra></extra>',
    visible="legendonly"
))

# Spot price line (already exists)
fig.add_vline(x=spot, line=dict(color="yellow", width=2, dash="dash"),
              annotation_text="Spot", annotation_position="top")

# Max OI diamonds with offset
offset = df_total["total_oi"].max() * 0.03

max_call_idx = df_total["oi_call"].idxmax()
max_put_idx  = df_total["oi_put"].idxmax()
max_total_idx = df_total["total_oi"].idxmax()

fig.add_trace(go.Scatter(
    x=[df_total.loc[max_call_idx, "strike"]],
    y=[df_total.loc[max_call_idx, "oi_call"] + offset],
    mode="markers",
    name="Max Call OI",
    marker=dict(color="green", size=12, symbol="diamond"),
    yaxis="y1",
    hovertemplate='🟢 Max Call OI: %{y:.0f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[df_total.loc[max_put_idx, "strike"]],
    y=[df_total.loc[max_put_idx, "oi_put"]],
    mode="markers",
    name="Max Put OI",
    marker=dict(color="red", size=12, symbol="diamond"),
    yaxis="y1",
    hovertemplate='🔴 Max Put OI: %{y:.0f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[df_total.loc[max_total_idx, "strike"]],
    y=[df_total.loc[max_total_idx, "total_oi"] - offset],
    mode="markers",
    name="Max Total OI",
    marker=dict(color="white", size=12, symbol="diamond"),
    yaxis="y1",
    hovertemplate='⚪ Max Total OI: %{y:.0f}<extra></extra>'
))

# Layout
fig.update_layout(
    title=f"{symbol} GEX Dashboard",
    xaxis_title="Strike Price",
    yaxis=dict(title="Net GEX / OI", side="left", color="white", showgrid=False),
    yaxis2=dict(title="Abs GEX", overlaying="y", side="right", color="white", showgrid=False),
    paper_bgcolor="black",
    plot_bgcolor="black",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, font=dict(color="white")),
    hovermode="x unified",
    barmode="group",
    font=dict(color="white")
)

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.01, y=0.99,
    text=(
        f"Total Abs Gamma: {total_gamma_abs:,.0f}<br>"
        f"Total Net Gamma: {total_net_gamma:,.0f}<br>"
        f"Bias: {'RANGE LIKELY (positive gamma)' if total_net_gamma > 0 else 'BREAKOUT LIKELY (negative gamma)'}"
    ),
    showarrow=False,
    font=dict(color="white", size=12),
    align="left",
    bgcolor="rgba(0,0,0,0.5)",
    bordercolor="white",
    borderwidth=1
)

# === Power Zone (mean reversion zone) ===
if "power_center" in globals() and power_center is not None:
    # Shaded band
    fig.add_vrect(
        x0=power_lower,
        x1=power_upper,
        fillcolor="purple",
        opacity=0.15,
        layer="below",
        line_width=0
    )
    # Power Center line
    fig.add_vline(
        x=power_center,
        line=dict(color="magenta", width=2, dash="dashdot"),
        annotation_text="Power Zone",
        annotation_position="top"
    )

fig.update_xaxes(range=[df["strike"].min(), df["strike"].max()])

fig.show()

In [ ]:
# 🔹 Cell 10: Price Magnet + Directional Bias + Predicted Close & Intraday Range

# Count red vs green Net GEX bars
num_green = (df_total["net_gex"] > 0).sum()
num_red   = (df_total["net_gex"] < 0).sum()

# Determine short-term bias
if num_green > num_red:
    direction_bias = "Slightly Bullish"
elif num_red > num_green:
    direction_bias = "Slightly Bearish"
else:
    direction_bias = "Neutral / Mixed"

# Gamma flip: pivot zone
gamma_flip_vals = df_total[df_total["net_gex"]*df_total["net_gex"].shift(1) < 0]["strike"]
gamma_flip = gamma_flip_vals.mean() if not gamma_flip_vals.empty else None

# Top Positive / Negative GEX (by net GEX)
pos_gex_df = df_total[df_total["net_gex"] > 0]
neg_gex_df = df_total[df_total["net_gex"] < 0]

top_positive_gex = pos_gex_df.loc[pos_gex_df["net_gex"].idxmax(), "strike"] if not pos_gex_df.empty else None
top_negative_gex = neg_gex_df.loc[neg_gex_df["net_gex"].idxmin(), "strike"] if not neg_gex_df.empty else None

# Second magnet (opposite side)
second_magnet = None
if direction_bias.lower().find("bear") >= 0 and top_positive_gex is not None:
    second_magnet = top_positive_gex
elif direction_bias.lower().find("bull") >= 0 and top_negative_gex is not None:
    second_magnet = top_negative_gex

# Top Abs GEX
top_abs_gex = df_total.loc[df_total["abs_gex"].idxmax(), "strike"] if not df_total.empty else None

# Total gamma (absolute) for range vs breakout
total_gamma_abs = df_total["abs_gex"].sum()

# -------------------------------
# Predicted close and intraday range
# -------------------------------
predicted_close = None
strikes = []
weights = []

# Use top positive and negative GEX for weighted prediction
for _, row in pos_gex_df.nlargest(2, "net_gex").iterrows():
    strikes.append(row["strike"])
    weights.append(row["abs_gex"])
for _, row in neg_gex_df.nsmallest(1, "net_gex").iterrows():
    strikes.append(row["strike"])
    weights.append(row["abs_gex"])

if strikes and weights:
    predicted_close = np.average(strikes, weights=weights)

# Estimate intraday range based on total gamma and strike span
if predicted_close is not None:
    strike_span = df_total["strike"].max() - df_total["strike"].min()
    # High total gamma → narrow range; low gamma → wider
    range_scale = max(0.05, 1 / (1 + total_gamma_abs/1e6))
    intraday_low  = predicted_close - strike_span*range_scale/2
    intraday_high = predicted_close + strike_span*range_scale/2

# -------------------------------
# Display summary
# -------------------------------
print("🔹 Price Magnet & Bias Analysis 🔹\n")
if gamma_flip is not None:
    print(f"Pivot / Gamma Flip Zone ≈ {gamma_flip:.2f}")

if direction_bias.lower().find("bear") >= 0:
    print(f"Downward Magnet (Top Negative GEX) is stronger → {top_negative_gex}")
elif direction_bias.lower().find("bull") >= 0:
    print(f"Upward Magnet (Top Positive GEX) is stronger → {top_positive_gex}")

if second_magnet is not None:
    print(f"Secondary Magnet (not as strong) → {second_magnet}")

if top_abs_gex is not None:
    print(f"Strike with Highest Hedging (price may be sticky) = {top_abs_gex}")

print(f"\nShort-term Directional Bias (based on GEX bars): {direction_bias}")
print(f"Green bars (positive GEX) = {num_green}, Red bars (negative GEX) = {num_red}")

# Range vs breakout note
if total_gamma_abs > 0:
    print(f"\nPrice is likely to stay in range (high total absolute gamma = {total_gamma_abs:,.0f}).")
else:
    print(f"\nPrice is more free to move / breakout likely (low total absolute gamma).")

# Predicted close & intraday range
if predicted_close is not None:
    print(f"\nPredicted Price Close ≈ {predicted_close:.2f}")
    print(f"Estimated Intraday Range ≈ {intraday_low:.2f} → {intraday_high:.2f}")

🔹 Price Magnet & Bias Analysis 🔹

Pivot / Gamma Flip Zone ≈ 655.00
Upward Magnet (Top Positive GEX) is stronger → 670.0
Secondary Magnet (not as strong) → 647.0
Strike with Highest Hedging (price may be sticky) = 660.0

Short-term Directional Bias (based on GEX bars): Slightly Bullish
Green bars (positive GEX) = 26, Red bars (negative GEX) = 14

Price is likely to stay in range (high total absolute gamma = 1,359,847,264).

Predicted Price Close ≈ 664.87
Estimated Intraday Range ≈ 663.90 → 665.85
